In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import seaborn as sns
import acquire
import prepare

In [2]:
df = acquire.get_sql_data()

In [5]:
df = acquire.cache_merged_data(cached=False)

In [8]:
df = prepare.prep_df(df)

In [13]:
# create a df consisting of the cohort_id == 0 observations
a = df[df.cohort_id==0]

In [15]:
# take the shape of this df
a.shape, df.shape

((52893, 11), (900223, 11))

*** 
#### NOTE: these observations are equal in amount the the NULL entries in our sequel Logs Table under cohort_id

* less than 6% of total observations
* Can't determing appropriate way to impute.  Filled with 0s during our preparation but may consider culling during certain exercises
***

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 to 2021-04-21
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        900223 non-null  datetime64[ns]
 1   path        900223 non-null  object        
 2   user_id     900223 non-null  int64         
 3   cohort_id   900223 non-null  int64         
 4   ip          900223 non-null  object        
 5   name        900223 non-null  object        
 6   start_date  900223 non-null  object        
 7   end_date    900223 non-null  object        
 8   created_at  900223 non-null  object        
 9   updated_at  900223 non-null  object        
 10  program_id  900223 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 82.4+ MB


1) Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [17]:
# Value counts of program_id
df.program_id.value_counts()

2    713365
3    103412
0     52893
1     30548
4         5
Name: program_id, dtype: int64

In [18]:
# Review program 4 observations
df[df.program_id==4]

,time,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id
date,,,,,,,,,,,
2018-03-22,2021-07-22 19:01:49,/,152,9,207.68.209.17,Apollo,2015-03-30 00:00:00,2015-07-29 00:00:00,2016-06-14 19:52:26,2016-06-14 19:52:26,4
2018-03-22,2021-07-22 19:01:54,content/html-css,152,9,207.68.209.17,Apollo,2015-03-30 00:00:00,2015-07-29 00:00:00,2016-06-14 19:52:26,2016-06-14 19:52:26,4
2018-03-22,2021-07-22 19:01:54,content/html-css/gitbook/images/favicon.ico,152,9,207.68.209.17,Apollo,2015-03-30 00:00:00,2015-07-29 00:00:00,2016-06-14 19:52:26,2016-06-14 19:52:26,4
2018-03-22,2021-07-22 19:02:47,content/html-css,152,9,207.68.209.17,Apollo,2015-03-30 00:00:00,2015-07-29 00:00:00,2016-06-14 19:52:26,2016-06-14 19:52:26,4
2018-03-22,2021-07-22 19:02:52,content/html-css/introduction.html,152,9,207.68.209.17,Apollo,2015-03-30 00:00:00,2015-07-29 00:00:00,2016-06-14 19:52:26,2016-06-14 19:52:26,4


***
#### Note: Program 4 only contains 5 observations, all occuring on the same day.  It doesn't appear to be associated with any viable CodeUp Program so going to exclude when exploring
***

In [39]:
# loop through all Programs grouping by program_id and pulling 7 largest path counts
pd.set_option('display.max_rows', None)
program_list = [1, 2, 3]

for p in program_list:
        
        prog_df = pd.DataFrame(df).groupby('program_id')['path'].value_counts()
        p_df = prog_df.loc[[p]]
        p_df = pd.DataFrame(p_df.nlargest(15))
        print(p_df)

                                                               path
program_id path                                                    
1          /                                                   1681
           index.html                                          1011
           javascript-i                                         736
           html-css                                             542
           spring                                               501
           java-iii                                             479
           java-ii                                              454
           java-i                                               444
           javascript-ii                                        429
           appendix                                             409
           jquery                                               344
           mysql                                                284
           content/html-css                     

***
#### Takeaways:
* Program IDs 1 & 2 appear to be Web Development programs while Program 3 is the Data Science program
* Comparing across the Web Development Programs 1 & 2:
    * javascript-i is the most accessed lesson
        * '/' is the most visited path across all programs but is not a curriculum page and is not considered
        * Again, index.html is the most visited path in Program 1 but is only referencing the Homepage not a curriculum page
    * html-css & java-iii are the next most visited curriculum pages across both cohorts 
    * NOTE: the spring, jquery and mysql curriculum pages were also highly visited across both Programs
* Looking at the Data Science program, Program 3:
    * classification/overview is the most visited curriculum and then followed by: 
        * 1-fundamentals/modern-data-scientist.jpg  1655
        * 1-fundamentals/AI-ML-DL-timeline.jpg      1651
        * 1-fundamentals/1.1-intro-to-data-science  1633
        * classification/scale_features_or_not.svg
***

4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [25]:
def acquire(file_name, column_names):
    return pd.read_csv(file_name, sep="\s", header=None, names=column_names, usecols=[0, 2, 3, 4, 5])

def prep(df, user):
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages = df['endpoint'].resample('d').count()
    return pages

def compute_pct_b(pages, span, weight, user):
    midband = pages.ewm(span=span).mean()
    stdev = pages.ewm(span=span).std()
    ub = midband + stdev*weight
    lb = midband - stdev*weight
    bb = pd.concat([ub, lb], axis=1)
    my_df = pd.concat([pages, midband, bb], axis=1)
    my_df.columns = ['pages', 'midband', 'ub', 'lb']
    my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

def find_anomalies(df, user, span, weight):
    pages = prep(df, user)
    my_df = compute_pct_b(pages, span, weight, user)
    # plt_bands(my_df, user)
    return my_df[my_df.pct_b>1]

#### Acquire Data

In [26]:
df4 = acquire('anonymized-curriculum-access-07-2021.txt', ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip'])

In [27]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018810 entries, 0 to 1018809
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   date       1018810 non-null  object 
 1   endpoint   1018809 non-null  object 
 2   user_id    1018810 non-null  int64  
 3   cohort_id  965313 non-null   float64
 4   source_ip  1018810 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 38.9+ MB


#### Explore 

In [28]:
prep(df4, 40).head()

date
2018-01-26     1
2018-01-27     0
2018-01-28     0
2018-01-29    14
2018-01-30     1
Freq: D, Name: endpoint, dtype: int64

#### Explore 

In [29]:
# Test the anomoly function on single user
user = 15
span = 30
weight = 3

anomalies = pd.DataFrame()
user_df = find_anomalies(df4, user, span, weight)
anomalies = pd.concat([anomalies, user_df], axis=0)

In [30]:
anomalies.head()

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2018-06-01,9,0.610958,7.457978,-6.236063,1.112605,15
2018-08-12,13,0.843729,10.605836,-8.918378,1.122625,15
2019-05-29,4,0.258065,3.256503,-2.740374,1.123981,15
2019-11-12,1,0.064527,0.814285,-0.685230,1.123850,15


In [31]:
# loop the anomalies function through all user ids and identify potential anomalies
span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df4.user_id.unique()):
    user_df = find_anomalies(df4, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [32]:
# set list max option at unlimited and count the count of pages and the frequency 
pd.set_option('display.max_rows', None)
anomalies.pages.value_counts(sort=False).tail(10)

128    1
129    1
146    1
153    1
165    1
179    1
192    1
198    1
272    1
343    1
Name: pages, dtype: int64

In [60]:
# Identify the user id associated with 3rd highest count
anomalies[anomalies.pages==198]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2018-06-02,198,16.484863,188.766965,-155.797239,1.026796,138


In [61]:
# Utilize the user id to identify other potential associated anomalous activity
anomalies[anomalies.user_id==138]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2018-06-02,198,16.484863,188.766965,-155.797239,1.026796,138
2018-12-15,3,0.193901,2.837208,-2.449406,1.030793,138
2019-06-09,4,0.258066,3.756252,-3.240120,1.034839,138
2019-09-16,5,0.323617,4.699978,-4.052743,1.034278,138


In [33]:
# Identify the user id associated with the 2nd highest count
anomalies[anomalies.pages==272]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-03-03,272,24.721632,266.780128,-217.336864,1.010782,341


In [33]:
# Identify the user id associated with the 2nd highest count
anomalies[anomalies.pages==272]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-03-03,272,24.721632,266.780128,-217.336864,1.010782,341


In [33]:
# Identify the user id associated with the highest count
anomalies[anomalies.pages==343]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2021-06-21,343,22.370564,322.155049,-277.413921,1.034767,804


In [34]:
# Utilize the user id to identify other potential associated anomalous activity
anomalies[anomalies.user_id==804]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2021-01-20,74,8.832557,73.838286,-56.173171,1.001244,804
2021-06-21,343,22.370564,322.155049,-277.413921,1.034767,804


***
#### Takeaways:
* In the case of our largest potential anomaly (343 count, user_id 804), it appears as though the occurence happens just after the cohort ended which was common when counts exceeded our upper Bollinger Band
    * The IP address used was in regular use beforehand.  No timestamps available to evaluate
    
* In the case of our 2nd largest potential anamoly (272 count, user_id 341), the information strongly corroborates anomalous behavior
    * over 5 unique IP addresses can be identified with the user ID
        * 91.105.19.58 is very similar to other user ids and suggests this may have been a centralized location for all cohort members
        * 173.174.243.231 appears to have been used regularly throughout the course dates suggesting a personal IP address
        * However, 204.44.122.76 ONLY appears on the date during which the 272 page count occurs.  173.174.243.231 also is active on this date and both appear to be iterating through the curriculum pages. 
            * Timestamps for both IPs a definitive
                * 173... iterates over several minutes
                * 204... multiple equivalent timestamps seperated by milliseconds suggesting the download automated/algo and not manual effort
        * 172.124.40.146 appears for the first time on 2/10/20 and visits an anomalous amount of pages (109)
            * this occurs several months after the cohort ends
            * wasn't used before and considering no activity leading up to this date, it would appear as though the user credentials were utilized by someone other than the previous student to download course curriculum
                * again the timestamps suggest it was an automated/algo and not manual effort

#### Appendix: Dataframes by user ID to explore all observations by user id

In [35]:
# pull all observations for user id 341.  Note: change max_rows number to 'None' to see all observations
pd.set_option('display.max_rows', 5)
df[df.user_id==341]

,time,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id
date,,,,,,,,,,,
2019-01-22,2021-07-22 15:23:24,/,341,29,97.105.19.58,Zion,2019-01-22 00:00:00,2019-06-04 00:00:00,2019-01-20 23:18:57,2019-01-20 23:18:57,2
2019-01-22,2021-07-22 15:25:51,toc,341,29,97.105.19.58,Zion,2019-01-22 00:00:00,2019-06-04 00:00:00,2019-01-20 23:18:57,2019-01-20 23:18:57,2
...,...,...,...,...,...,...,...,...,...,...,...
2021-02-10,2021-07-22 08:31:29,appendix/further-reading/spring/pagination,341,29,172.124.70.146,Zion,2019-01-22 00:00:00,2019-06-04 00:00:00,2019-01-20 23:18:57,2019-01-20 23:18:57,2
2021-02-10,2021-07-22 08:31:38,appendix/further-reading/spring/seeder,341,29,172.124.70.146,Zion,2019-01-22 00:00:00,2019-06-04 00:00:00,2019-01-20 23:18:57,2019-01-20 23:18:57,2


In [36]:
# pull all observations for user id 138.  Note: change max_rows number to 'None' to see all observations
pd.set_option('display.max_rows', 5)
df[df.user_id==138]

,time,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id
date,,,,,,,,,,,
2018-03-09,2021-07-22 09:33:19,slides/annotations,138,22,97.105.19.61,Teddy,2018-01-08 00:00:00,2018-05-17 00:00:00,2018-01-08 13:59:10,2018-01-08 13:59:10,2
2018-03-09,2021-07-22 09:36:13,java-ii/annotations,138,22,97.105.19.61,Teddy,2018-01-08 00:00:00,2018-05-17 00:00:00,2018-01-08 13:59:10,2018-01-08 13:59:10,2
...,...,...,...,...,...,...,...,...,...,...,...
2019-09-16,2021-07-22 20:58:37,1-fundamentals/AI-ML-DL-timeline.jpg,138,22,108.65.244.91,Teddy,2018-01-08 00:00:00,2018-05-17 00:00:00,2018-01-08 13:59:10,2018-01-08 13:59:10,2
2019-09-16,2021-07-22 20:58:37,1-fundamentals/modern-data-scientist.jpg,138,22,108.65.244.91,Teddy,2018-01-08 00:00:00,2018-05-17 00:00:00,2018-01-08 13:59:10,2018-01-08 13:59:10,2


In [38]:
# pull all observations for user id 804.  Note: change max_rows number to 'None' to see all observations
pd.set_option('display.max_rows', 5)
df[df.user_id==804]

,time,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id
date,,,,,,,,,,,
2020-11-03,2021-07-22 10:15:51,javascript-i,804,132,69.91.64.132,Kalypso,2020-11-02 00:00:00,2021-05-04 00:00:00,2020-11-02 20:43:58,2020-11-02 20:43:58,2
2020-11-04,2021-07-22 09:00:55,javascript-i,804,132,69.91.64.132,Kalypso,2020-11-02 00:00:00,2021-05-04 00:00:00,2020-11-02 20:43:58,2020-11-02 20:43:58,2
...,...,...,...,...,...,...,...,...,...,...,...
2021-04-16,2021-07-22 11:31:41,spring,804,132,66.69.1.31,Kalypso,2020-11-02 00:00:00,2021-05-04 00:00:00,2020-11-02 20:43:58,2020-11-02 20:43:58,2
2021-04-16,2021-07-22 11:31:44,spring/extra-features/file-upload,804,132,66.69.1.31,Kalypso,2020-11-02 00:00:00,2021-05-04 00:00:00,2020-11-02 20:43:58,2020-11-02 20:43:58,2
